# Regresion Logistica: Deteccion de SPAM

En este ejercicio se muestran los fundamentos de regresion Logistica planteando uno de los primeros problemas que fueron solucionados mediante el uso de tecnicas de Machine Learning


## Enunciado del ejercicio:

se propone la construccion de un sistema de aprendizaje automatico capz de la tierra de predecir si un correo determinado se corresponde con un correo SPAM o no, para ello se utilizara el siguiente conjunto de datos:

###### [2007 TREC Public SPAM Corpus](https://plg.uwaterloo.ca/~gvcormac/treccorpus07/)

The corpus trec07p contains 75,419 messages:

    25,220 HAM 
    50,190 SPAM

These messages  constitute all the message delivered to a particular server between these dates:

    Sun, 8 Apr 2007 13:07:21 -0400
    Fri, 6 Jul 2007 07:04:53 -0400



## 1.- Funciones Complementarias

En este caso practico relacionado con la deteccion de de correos electronicos de SPAM, el conjunto de datos que se dispone esta formado por correos electronicos con sus correspondientes cabeceras y campos adicionales. Por lo tanto requieren un preprocesamiento previo a que sean ingeridos por el algoritmo de Machine Learning (ML).

In [7]:
## En esta clase facilita el preprocesamiento de correos electronicos que poseen codigo html
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs = True
        self.fed = []

    def handle_data(self, d):
        self.fed.append(d)

    def get_data(self):
        return ''.join(self.fed)

In [9]:
# Esta funcion se encarga de eliminar los tags HTML que se encuentren en el texto del correo electronico.
def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [15]:
# Ejemplo de eliminacion de los tags HTML de un texto.
t = '<tr><td align="left"><a href="../../issues/51/16.html#article">Phrack World News</a></td></tr>'
strip_tags(t)

'Phrack World News'

Ademas de eliminar los posibles tags HTML que se encuentren en el correo electronico deben realizarse otras acciones de preprocesamiento, para evitar que los mensajes contengan ruido incesario. Entre ellos se encuentra la elimincacion de los signos de puntuacion, eliminacion de posibles campos de correo electronico que no son relevantes o eliminacion de los afijos de una palabra manteniendo unicamente la raiz de la misma (Stemming). La clase que se muestra a continuacion realiza estas transformaciones

In [25]:
import email 
import string
import nltk

class Parser:
    def __init__(self):
        self.stemmer = nltk.PorterStemmer()
        self.stopwords = set(nltk.corpus.stopwords.words('english'))
        self.punctuation = lis(string.punctuation)

    def parse(self, email_path):
        """Parse and email."""
        with open(email_path, error = 'ignore') as e:
            msg = email.message_from_file(e)
        return None if not msg else self.get_email_content(msg)

    def get_email_content(self, msg):
        """Extract the email content."""
        subject = self.tokenize(msg['Subject']) if msg['Subject'] else []
        body = self.get_email_body(msg.get_payload(), 
                                  msg.get_content_type())
        content_type= msg.get_content_type()
        # Returning the content ot the email
        return {"subject": subject,
               "body": body,
               "content_type": content_type}

    def get_email_body(self, payload, content_type):
        """Extract the body of the email."""
        body = []
        if type(payload) is str and content_type == 'text/plain':
            return self.tokenize(payload)
        elif type(payload) is str and content_type == 'text/html':
            return self.tokenize(strip_tags(payload))
        elif type(payload) is list:
            for p in payload:
                body += self.get_email_body(p.get_payload(),
                                            p.get_content_type())
        return body